In [ ]:
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# # Gabungkan semua token menjadi satu teks
# all_words = " ".join([" ".join(tokens) for tokens in df["tokens_rare_removed"]])

# # Buat WordCloud
# wordcloud = WordCloud(
#     width=800, height=400, background_color="white", colormap="viridis"
# ).generate(all_words)

# # Tampilkan WordCloud
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")  # Hilangkan sumbu
# plt.title("WordCloud dari Review yang Telah Dibersihkan")
# plt.show()